In [1]:
class Student:
    # This keeps track of what ID to give the next student
    next_id = 1

    def __init__(self, name, subjects=None):
        self.id = Student.next_id
        Student.next_id += 1
        self.name = name
        self.subjects = subjects if subjects else {}

    def add_subject(self, subject, score):
        try:
            score = float(score)
            if score < 0 or score > 100:
                raise ValueError("Score must be between 0 and 100")
            self.subjects[subject] = score
            return True
        except ValueError as e:
            print(f"Error: {e}")
            return False

    def calculate_average(self):
        if not self.subjects:
            return 0
        return sum(self.subjects.values()) / len(self.subjects)

    def get_grade(self):
        average = self.calculate_average()
        if average >= 90:
            return "A"
        elif average >= 80:
            return "B"
        elif average >= 70:
            return "C"
        elif average >= 60:
            return "D"
        else:
            return "F"

    def to_dict(self):
        return {
            "id": self.id,
            "name": self.name,
            "subjects": self.subjects
        }

    @classmethod
    def from_dict(cls, data):
        student = cls(data["name"], data["subjects"])
        student.id = data["id"]

        if student.id >= cls.next_id:
            cls.next_id = student.id + 1

        return student

In [2]:
import json

class GradeManager:
    def __init__(self):
        self.students = []

    def add_student(self):
        name = input("Enter student name: ").strip()
        if not name:
            print("Error: Name cannot be empty.")
            return

        student = Student(name)

        while True:
            subject = input("Enter subject name (or 'done' to finish): ").strip()
            if subject.lower() == 'done':
                break
            if not subject:
                print("Error: Subject name cannot be empty.")
                continue

            try:
                score = float(input(f"Enter score for {subject}: "))
                if score < 0 or score > 100:
                    print("Error: Score must be between 0 and 100.")
                    continue
                student.subjects[subject] = score
            except ValueError:
                print("Error: Please enter a valid number for the score.")

        self.students.append(student)
        print(f"Student '{name}' added successfully with ID: {student.id}")

    def find_student_by_id(self, student_id):
        for student in self.students:
            if student.id == student_id:
                return student
        return None

    def update_scores(self):
        try:
            student_id = int(input("Enter student ID to update: "))
        except ValueError:
            print("Error: Please enter a valid numeric ID.")
            return

        student = self.find_student_by_id(student_id)
        if not student:
            print(f"Error: Student with ID {student_id} not found.")
            return

        print(f"Updating scores for {student.name} (ID: {student.id})")
        print("Current subjects and scores:")
        for subject, score in student.subjects.items():
            print(f"  {subject}: {score}")

        while True:
            subject = input("Enter subject name to update (or 'done' to finish): ").strip()
            if subject.lower() == 'done':
                break

            if subject not in student.subjects:
                add_new = input(f"Subject '{subject}' not found. Add as new subject? (y/n): ").strip().lower()
                if add_new != 'y':
                    continue

            try:
                score = float(input(f"Enter new score for {subject}: "))
                if score < 0 or score > 100:
                    print("Error: Score must be between 0 and 100.")
                    continue
                student.subjects[subject] = score
                print(f"Updated {subject}: {score}")
            except ValueError:
                print("Error: Please enter a valid number for the score.")

        print("Scores updated successfully.")

    def view_report(self):
        try:
            student_id = int(input("Enter student ID to view report: "))
        except ValueError:
            print("Error: Please enter a valid numeric ID.")
            return

        student = self.find_student_by_id(student_id)
        if not student:
            print(f"Error: Student with ID {student_id} not found.")
            return

        print("\n" + "="*50)
        print(f"STUDENT REPORT - {student.name} (ID: {student.id})")
        print("="*50)

        if not student.subjects:
            print("No subjects found for this student.")
            return

        for subject, score in student.subjects.items():
            print(f"{subject}: {score}")

        average = student.calculate_average()
        grade = student.get_grade()

        print("-"*50)
        print(f"Average: {average:.2f}")
        print(f"Grade: {grade}")
        print("="*50)

    def delete_student(self):
        try:
            student_id = int(input("Enter student ID to delete: "))
        except ValueError:
            print("Error: Please enter a valid numeric ID.")
            return

        student = self.find_student_by_id(student_id)
        if not student:
            print(f"Error: Student with ID {student_id} not found.")
            return

        self.students = [s for s in self.students if s.id != student_id]
        print(f"Student '{student.name}' (ID: {student_id}) deleted successfully.")

    def save_to_file(self, filename="grades.json"):
        try:
            data = {
                "students": [student.to_dict() for student in self.students],
                "next_id": Student.next_id
            }

            with open(filename, 'w') as file:
                json.dump(data, file, indent=4)

            print(f"Data saved successfully to {filename}")
        except Exception as e:
            print(f"Error saving data: {e}")

    def load_from_file(self, filename="grades.json"):
        try:
            with open(filename, 'r') as file:
                data = json.load(file)

            self.students = [Student.from_dict(student_data) for student_data in data["students"]]
            Student.next_id = data["next_id"]

            print(f"Data loaded successfully from {filename}")
            print(f"Loaded {len(self.students)} student records")
        except FileNotFoundError:
            print(f"File {filename} not found. Starting with empty records.")
        except Exception as e:
            print(f"Error loading data: {e}")

In [3]:
def display_menu():
    print("\n" + "="*40)
    print("    STUDENT REPORT CARD MANAGER")
    print("="*40)
    print("1. Add Student")
    print("2. Update Scores")
    print("3. View Report")
    print("4. Delete Student")
    print("5. Save Data")
    print("6. Load Data")
    print("7. Exit")
    print("="*40)

def main():
    manager = GradeManager()

    while True:
        display_menu()

        try:
            choice = int(input("Enter your choice (1-7): "))
        except ValueError:
            print("Error: Please enter a valid number.")
            continue

        if choice == 1:
            manager.add_student()
        elif choice == 2:
            manager.update_scores()
        elif choice == 3:
            manager.view_report()
        elif choice == 4:
            manager.delete_student()
        elif choice == 5:
            manager.save_to_file()
        elif choice == 6:
            manager.load_from_file()
        elif choice == 7:
            print("Exiting the application. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 7.")

In [4]:
from google.colab import files
import os

def download_grades_file():
    if os.path.exists("grades.json"):
        files.download("grades.json")
        print("Downloaded grades.json file")
    else:
        print("No grades.json file found")

def upload_grades_file():
    uploaded = files.upload()
    for filename in uploaded.keys():
        if filename == "grades.json":
            print("Uploaded grades.json successfully")
        else:
            print(f"Uploaded {filename}, but please name it grades.json for the system to recognize it")

def view_json_content():
    try:
        with open("grades.json", "r") as file:
            content = file.read()
            print("Content of grades.json:")
            print("=" * 50)
            print(content)
    except FileNotFoundError:
        print("No grades.json file found. Save some data first.")

In [5]:
if __name__ == "__main__":
    main()


    STUDENT REPORT CARD MANAGER
1. Add Student
2. Update Scores
3. View Report
4. Delete Student
5. Save Data
6. Load Data
7. Exit
Enter your choice (1-7): 1
Enter student name: Arun
Enter subject name (or 'done' to finish): Maths
Enter score for Maths: 98
Enter subject name (or 'done' to finish): Science
Enter score for Science: 85
Enter subject name (or 'done' to finish): English
Enter score for English: 75
Enter subject name (or 'done' to finish): done
Student 'Arun' added successfully with ID: 1

    STUDENT REPORT CARD MANAGER
1. Add Student
2. Update Scores
3. View Report
4. Delete Student
5. Save Data
6. Load Data
7. Exit
Enter your choice (1-7): 3
Enter student ID to view report: 1

STUDENT REPORT - Arun (ID: 1)
Maths: 98.0
Science: 85.0
English: 75.0
--------------------------------------------------
Average: 86.00
Grade: B

    STUDENT REPORT CARD MANAGER
1. Add Student
2. Update Scores
3. View Report
4. Delete Student
5. Save Data
6. Load Data
7. Exit
Enter your choice (1-7

In [6]:
view_json_content()

Content of grades.json:
{
    "students": [
        {
            "id": 1,
            "name": "Arun",
            "subjects": {
                "Maths": 98.0,
                "Science": 85.0,
                "English": 75.0
            }
        },
        {
            "id": 2,
            "name": "NXT wave",
            "subjects": {
                "Maths": 90.0,
                "Science": 95.0,
                "English": 60.0
            }
        }
    ],
    "next_id": 3
}
